In [3]:
import numpy as np
from astropy.table import Table
from astropy.io.ascii import InconsistentTableError
from astropy.io.fits import getheader
import matplotlib.pyplot as plt
from scipy import stats
import os
from glob import glob
from scipy.optimize import least_squares
from tqdm import tqdm_notebook

In [4]:
def model2(theta, mag, y, zp=22.5, sigdet=5, k=1):
    a, b, maglim = theta 
    teff = np.exp(a + b * (maglim - 21.)) 

    # Compute flux/limit. 
    F = 10**(-0.4 * (mag - zp)) 
    Flim = 10**(-0.4 * (maglim - zp)) 

    # Compute noise. 
    Fnoise = (Flim / sigdet)**2 * k * teff - Flim
    magerr = 2.5 / np.log(10.) * np.sqrt((1. + Fnoise / F) / (F * k * teff))

    return np.sum(np.abs(y - magerr))
    #return magerr - y

In [5]:
datapath = f'{os.environ["HOME"]}/Projects/planckClusters/data/proc2'
files = glob(f'{datapath}/PSZ**/*K_cal.cat')

In [6]:
files

['/home/boada/Projects/planckClusters/data/proc2/PSZ1_G148.20+23.49/PSZ1_G148.20+23.49K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ1_G142.17+37.28/PSZ1_G142.17+37.28K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ2_G137.58+53.88/PSZ2_G137.58+53.88K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ1_G286.25+62.68/PSZ1_G286.25+62.68K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ1_G031.91+67.94/PSZ1_G031.91+67.94K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ1_G244.48+34.06/PSZ1_G244.48+34.06K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ1_G096.44-10.40/PSZ1_G096.44-10.40K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ2_G071.67-42.76/PSZ2_G071.67-42.76K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ1_G105.91-38.39/PSZ1_G105.91-38.39K_cal.cat',
 '/home/boada/Projects/planckClusters/data/proc2/PSZ1_G108.90-52.04/PSZ1_G108.90-52.04K_cal.cat',
 '/home/boada/Projec

In [7]:
filters = ['g', 'r', 'i', 'z', 'K']
limits = {}
for filt in tqdm_notebook(filters, desc='filters'):
    maglim = []
    for f in tqdm_notebook(files, desc='files', leave=False):
        new_f = f'{f[:-9]}{filt}_cal.cat'        
        if os.path.isfile(new_f):
            f = new_f
        else:
            maglim.append(-1)
            continue
        cat = Table.read(f, format='ascii')
        # filter out non-detections
        cat = cat[cat['MAG_AUTO'] < 30]
        # filter out the stars
        cat = cat[cat['CLASS_STAR']< 0.8]
        if len(cat) < 100:
            maglim.append(-1)
        else:
            fits_file = f'{f[:-8]}.fits'
            hdr = getheader(fits_file)
            zp = hdr['magzero']
            result = least_squares(model2, (4, 1, 20), loss='cauchy', f_scale=np.std(cat['MAGERR_AUTO']),
                           args=(cat['MAG_AUTO'], cat['MAGERR_AUTO']), kwargs={'zp':zp, 'sigdet':3}, 
                           bounds=([3,0.5, 15], [5,2, 30]))
            maglim.append(result['x'][2])
    limits[filt] = maglim

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(limits)

In [15]:
df.describe()

,g,r,i,z,K
count,80.000000,80.000000,80.000000,80.000000,80.000000
mean,16.249540,16.059091,16.026271,15.470404,21.318638
std,11.380659,11.255119,11.226700,11.181487,0.681492
min,-1.000000,-1.000000,-1.000000,-1.000000,18.801006
25%,-1.000000,-1.000000,-1.000000,-1.000000,20.814546
50%,23.446769,23.187573,23.237844,22.704416,21.491795
75%,23.919518,23.709339,23.555344,23.126283,21.823434
max,24.669797,24.345082,24.175711,24.127846,22.167039


In [16]:
df.replace(-1, np.nan).describe()

,g,r,i,z,K
count,56.000000,56.000000,56.000000,55.000000,80.000000
mean,23.642200,23.370131,23.323245,22.956951,21.318638
std,0.743882,0.738106,0.569614,0.481309,0.681492
min,20.533843,20.048669,21.216852,21.450051,18.801006
25%,23.377925,23.114832,23.107482,22.668354,20.814546
50%,23.761087,23.465553,23.366265,22.992246,21.491795
75%,24.051688,23.797146,23.699036,23.216523,21.823434
max,24.669797,24.345082,24.175711,24.127846,22.167039


In [17]:
fields = [f'{f.rsplit("/")[-1][:-9]}' for f in files]

In [18]:
df['fields'] = fields

In [19]:
df

,g,r,i,z,K,fields
0,23.552037,23.238198,23.253262,23.064814,21.928263,PSZ1_G148.20+23.49
1,23.462874,23.121495,23.233203,23.215255,21.954919,PSZ1_G142.17+37.28
2,23.733361,23.391225,23.486331,22.975911,21.681539,PSZ2_G137.58+53.88
3,23.333869,23.336319,23.427244,22.946449,22.118406,PSZ1_G286.25+62.68
4,23.603999,23.494681,23.460437,22.948528,22.065140,PSZ1_G031.91+67.94
5,-1.000000,-1.000000,-1.000000,-1.000000,21.324506,PSZ1_G244.48+34.06
6,23.307893,23.014027,23.315319,23.391770,21.996544,PSZ1_G096.44-10.40
7,24.032375,23.783068,23.677466,23.034849,22.066333,PSZ2_G071.67-42.76
8,24.088340,23.667774,23.517554,22.971916,22.090588,PSZ1_G105.91-38.39
9,23.907309,23.857055,23.826679,-1.000000,21.482872,PSZ1_G108.90-52.04


In [20]:
df = df.replace(-1, np.nan)

In [22]:
df.to_csv('./psz_depths.csv', index=False)